In [ ]:
# plt.show()で可視化されない人はこのセルを実行してください。
%matplotlib inline

#  深層学習のチューニング

- **[2.1 ハイパーパラメータ](#2.1-ハイパーパラメータ)**
    - **[2.1.1 ハイパーパラメータ](#2.1.1-ハイパーパラメータ)**
<br><br>
- **[2.2 ネットワーク構造](#2.2-ネットワーク構造)**
    - **[2.2.1 ネットワーク構造](#2.2.1-ネットワーク構造)**
<br><br>
- **[2.3 ドロップアウト](#2.3-ドロップアウト)**
    - **[2.3.1 ドロップアウト](#2.3.1-ドロップアウト)**
<br><br>
- **[2.4 活性化関数](#2.4-活性化関数)**
    - **[2.4.1 活性化関数](#2.4.1-活性化関数)**
    - **[2.4.2 シグモイド関数](#2.4.2-シグモイド関数)**
    - **[2.4.3 ReLU](#2.4.3-ReLU)**
<br><br>
- **[2.5 損失関数](#2.5-損失関数)**
    - **[2.5.1 損失関数](#2.5.1-損失関数)**
    - **[2.5.2 二乗誤差](#2.5.2-二乗誤差)**
    - **[2.5.3 クロスエントロピー誤差](#2.5.3-クロスエントロピー誤差)**
<br><br>
- **[2.6 最適化関数](#2.6-最適化関数)**
    - **[2.6.1 最適化関数](#2.6.1-最適化関数)**
<br><br>
- **[2.7 学習率](#2.7-学習率)**
    - **[2.7.1 学習率](#2.7.1-学習率)**
<br><br>
- **[2.8 ミニバッチ学習](#2.8-ミニバッチ学習)**
    - **[2.8.1 ミニバッチ学習](#2.8.1-ミニバッチ学習)**
<br><br>
- **[2.9 反復学習](#2.9-反復学習)**
    - **[2.9.1 反復学習](#2.9.1-反復学習)**
<br><br>
- **[2.10 まとめ問題(提出不要)](#2.10-まとめ問題(提出不要))**

***

## 2.1 ハイパーパラメータ

### 2.1.1 ハイパーパラメータ

ニューラルネットワークモデルには、ネットワークを構成する際に調整が必要となる<b style='color: #AA0000'>ハイパーパラメータ</b>と呼ばれるものがいくつか存在します。

ハイパーパラメータは数多く存在し、適切に設定しないと正しく学習が行われません。そこで、新規モデル作成時には最適なハイパーパラメータを設計する必要があります。<br>

実際のコードでハイパーパラメータを見てみましょう。

```python
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
# ハイパーパラメータ：活性化関数
model.add(Activation("sigmoid"))
# ハイパーパラメータ：隠れ層の数、隠れ層のユニット数
model.add(Dense(128))
model.add(Activation("sigmoid"))
# ハイパーパラメータ：ドロップアウトする割合（rate）
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

# ハイパーパラメータ：学習率（lr）
sgd = optimizers.SGD(lr=0.01)

# ハイパーパラメータ：最適化関数（optimizer）
# ハイパーパラメータ：誤差関数（loss）
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

# ハイパーパラメータ：バッチサイズ（batch_size）
# ハイパーパラメータ：エポック数（epochs）
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))
```

上記は、<a href='https://aidemy.net/courses/5090/exercises/H1PL928jIxf' target='_blank'>Chapter1</a>のMNIST分類のコードに少しだけ変更を加え、いくつかのパラメータを明示したものです。なお、`metrics`は評価関数です。評価関数については<a href='https://aidemy.net/courses/2010' target='_blank'>機械学習概論</a>を参照してください。

このChapterではそれぞれのハイパーパラメータの意味を理解し、ネットワークの構築、調整の方法を学んでいきましょう。

#### 問題

- ハイパーパラメータについて説明した文として正しいものを選んでください。

- ハイパーパラメータは学習時にモデルが自動的に調整する。
- ハイパーパラメータは自分で調整する必要がある。
- ハイパーパラメータは適切に設定するのが良いが、適切でなくとも多くの場合問題なく学習は進行する。

#### ヒント

- パラメーターのうち人が調整するパラメーターをハイパーパラメーターと言います。

#### 解答

- ハイパーパラメータは自分で調整する必要がある。

***

## 2.2 ネットワーク構造

### 2.2.1 ネットワーク構造の設定

入力層と出力層の間にある **隠れ層** の数やユニット数は自由に指定することが可能で、数を多くすることによって多彩な関数を表現できるようになります。<br>

しかし、隠れ層の数が多い場合は、重みの調整の難易度が上がって **学習の進行が遅く** なったり、ユニット数が多い場合は、重要度の低い特徴量を抽出して **過学習** (汎化性能が低い状態)を起こしやすくなったりします。
したがって、ただやみくもに数を増やすのではなく、学習に適切な数を設定することが必要です。

同じような実装例を参考にするなど前例に基づいてネットワーク構造を検討します。

#### 問題

隠れ層の構造がモデルの学習に与える影響を確認し、次の3つの中から一番精度の高いモデルを予想してみましょう。

 - A: ユニット数256の全結合隠れ層1つ、ユニット数128の全結合隠れ層1つ
 - B: ユニット数256の全結合隠れ層1つ、ユニット数128の全結合隠れ層3つ
 - C: ユニット数256の全結合隠れ層1つ、ユニット数1568の全結合隠れ層1つ

コード内の指示にしたがって、コードを変更してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))

def funcA():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcB():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcC():
    model.add(Dense(1568))
    model.add(Activation("sigmoid"))

#A、B、Cのモデルの中から1つを選び、残りの2つはコメントアウトしてください。
#---------------------------
funcA()
funcB()
funcC()
#---------------------------

model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 全パターン試してみてください。
- evaluate acc: が精度を表しています。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))

def funcA():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcB():
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))
    model.add(Dense(128))
    model.add(Activation("sigmoid"))

def funcC():
    model.add(Dense(1568))
    model.add(Activation("sigmoid"))

# A、B、Cのモデルの中から1つを選び、残りの2つはコメントアウトしてください。
#---------------------------
funcA()
#funcB()
#funcC()
#---------------------------

model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.3 ドロップアウト

### 2.3.1 ドロップアウト

<b style='color: #AA0000'>ドロップアウト</b>は、 **学習データに対する過学習を防ぎ、モデルの精度をあげるための手法の一つ** です。  

ドロップアウトではランダムにニューロンを削除（0で上書き）しながら、学習を繰り返します。それによって、ニューラルネットワークは特定のニューロンに依存することなく、より **汎用的な特徴を学習する** ようになります。 

ドロップアウトの記述は下記のようになります。
```python
model.add(Dropout(rate=0.5))
```
rateは削除するユニットの割合です。

**ドロップアウトの位置やrateについては、どちらもハイパーパラメータとなります。**

#### 問題

ドロップアウトを実装し、訓練データとテストデータ、それぞれの正解率を近づけましょう。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))

# ---------------------------
# ここに先ほどのドロップアウトのコードを記述します

# ---------------------------

model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

#### ヒント

- `Dropout()`を使用してドロップアウトを実装します。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))

#- --------------------------
# ここを書いて下さい
model.add(Dropout(rate=0.5))
# ここまで書いて下さい
# ---------------------------
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

***

## 2.4 活性化関数

### 2.4.1 活性化関数

<b style='color: #AA0000'>活性化関数</b>とは、全結合層などの後に適用する関数で、ニューロンの**発火**に相当するものです。  

全結合層では、入力を線形変換して出力しますが、 **活性化関数を用いることで非線形性をもたせることができます** 。

活性化関数を使用しないと、下図のように**一本の直線で分離できず（線形分離不可能）、データが分類できなくなる** のです。

<img src='https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_10.png'>

**<center>図2.4.1-1 線形分離不可能な散布図</center>**

活性化関数によって非線形性をもたせることで、 **線形分離不可能なモデルでも、適切に学習が進めば必ず分類できるようになります** 。

なお、活性化関数にもいくつかの種類があるので、適切なものを選ぶことが大切です。 

#### 問題

- 活性化関数を使う理由として正しい選択肢を選んでください。

- モデルに線形性をもたせ、線形分離可能なデータに対応させるため。
- モデルに線形性をもたせ、線形分離不可能なデータに対応させるため。
- モデルに非線形性をもたせ、線形分離可能なデータに対応させるため。
- モデルに非線形性をもたせ、線形分離不可能なデータに対応させるため。

#### ヒント

- モデルが線型性の場合は、線形分離不可能なデータを分類できません。

#### 解答

- モデルに非線形性をもたせ、線形分離不可能なデータに対応させるため。

***

### 2.4.2 シグモイド関数

<b style='color: #AA0000'>シグモイド関数</b>は活性化関数のひとつで、入力値を0から1の間の数値にして出力します。下記のような式になります。

$$
sigmoid(x) = \frac{1}{1+e^{-x}}
$$

<img src='https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_20.png'>

**<center>図2.4.2-1 シグモイド関数</center>**

青いグラフが **シグモイド関数** で、オレンジ色のグラフがシグモイド関数を微分した **導関数** です。

#### 問題

- 説明文のグラフからわかる、シグモイド関数の説明として正しいものを1つ選んでください。

- 出力は必ず区間 `(0,1)` に収まるので、極端な出力値が少ない。
- どのような区間にも収まらず、極端な出力値が生成される可能性がある。 
- 出力が広い値をとるので、学習速度が早くなる。
- 出力の範囲が限られていないので、学習速度が遅くなる。

#### ヒント

- 縦軸の値に注目しましょう。

#### 解答

出力は必ず区間 `(0,1)` に収まるので、極端な出力値が少ない。

***

### 2.4.3 ReLU

よく使われる活性化関数として <b style='color: #AA0000'>ReLU関数（ランプ関数）</b>があります。この関数は入力する値が0以下であれば0を、1以上であれば値をそのまま入力値を出力します。 
ReLUは、**Rectified Linear Unit**の略で、下記のような式となります。

$$
\mathrm{ReLU}(x) = \begin{cases} 0 (x<0) \\ x (x\geq 0)\end{cases}
$$

<img src='https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_30.png'>

**<center>図2.4.3-1 ReLU関数</center>**

青いグラフが **ReLU関数** で、オレンジ色のグラフが **導関数** です。

#### 問題

- 説明文のグラフからわかる、ReLUの説明として正しいものを1つ選んでください。

- 出力は必ず区間(0,1)に収まるので、極端な出力値が少ない。
- 出力はどのような区間にも収まらず、極端な出力値が生成されうる。
- 出力が広い値をとるので、学習速度が遅くなる。
- 出力の範囲が限られているので、学習速度が遅くなる。

#### ヒント

- 一般的に、出力が大きい値だと学習速度は早くなります。

#### 解答

- 出力はどのような区間にも収まらず、極端な出力値が生成されうる。

***

## 2.5 損失関数

### 2.5.1 損失関数

学習時に出力データと教師データとの差を評価する関数を <b style='color: #AA0000'>損失関数（誤差関数）</b>といいます。

正解率を指標に評価することも可能ですが、それだけでは個々の出力データの正解・不正解といった細かい結果までは分かりません。<br>
つまり、一つひとつの出力データと教師データの差をみるために **損失関数** が使われるのです。

損失関数にはたくさんの種類がありますが、機械学習によく使われるものとしては **二乗誤差** や **クロスエントロピー誤差** などが挙げられます。<br>
この2つについては、後ほど詳しく説明します。

また、損失関数の微分の計算を効率的にするために誤差逆伝播法という手法が使われます。この手法では、出力データと教師データの差を **最小化** するように各層の **重みを更新** します。

#### 問題

 - 損失関数の性質を示した文として適切なものを選んでください。

- 一般に、損失関数を最大化するように各層の重みを更新する。
- 損失関数は重みを更新する際に重要な役割を持つので、適切なものを選ぶ必要がある。
- 損失関数には正解率を求める式をそのまま使うのが良い。
- 損失関数は1種類しかないので、これはハイパーパラメータではない。

#### ヒント

- 損失関数には様々な種類が存在し、値を最小化するように重みの更新をします。

#### 解答

- 損失関数は重みを更新する際に重要な役割を持つので、適切なものを選ぶ必要がある。

***

### 2.5.2 二乗誤差

<b style='color: #AA0000'>平均平方二乗誤差</b>は最小二乗法と並んで、統計学などの分野でよく使用される誤差関数です。
下記の式で記述します。

$$
{E} = \sum_{i=1}^N ({t_i - y_i})^{2}
$$

<img src='https://aidemyexstorage.blob.core.windows.net/aidemycontents/1543300574968104.png' width=400>

<b><center>誤差 = (<b style='color: #AA0000'>2</b>- <b style='color: 0000AA'>3</b>$)^2$ + (<b style='color: #AA0000'>4</b>- <b style='color: #0000AA'>5</b>$)^2$ + (<b style='color: #AA0000'>6</b>- <b style='color: #0000AA'>8</b>$)^2$ = 6</center></b>

<b><center>図2.5.2-1　二乗誤差</center></b>

$y_i$は **予測ラベル** 、$t_i$は **正解ラベル** です。平均平方二乗誤差は連続値の評価を得意とするため、 **主に回帰モデルの誤差関数** として使われます。

#### 問題

- 平均平方二乗誤差の説明として正しいものを1つ選んでください。

- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい
- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しにくい
- 分類に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい
- 分類に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しにくい

#### ヒント

- 下に凸の放物線をイメージしてください。

#### 解答

- 回帰に向いており、最小値の付近ではゆっくりと更新が行われるため、学習が収束しやすい

***

### 2.5.3 クロスエントロピー誤差

<b style='color: #AA0000'>クロスエントロピー誤差</b>は、 **分類の評価に特化** しているため、主に **分類モデルの誤差関数** として使われます。<br>
式は下記の通りになります。<br>

$$
E=\sum_{i=1}^N (-t_i\log y_i-(1-t_i)\log (1-y_i))
$$

<br>

また、一般的には以下の式が使用されることが多いです。<br>


$$
E=-\sum_{i=1}^N t_i\log y_i
$$

<br>

そうすることによって、ラベルが`1`以外の項はすべて0になるため、<b>実質的に正解ラベルの誤差のみを計算していることになります。</b><br>

<figure>
    <img src='https://aidemyexstorage.blob.core.windows.net/aidemycontents/1561008430395325.png' width=500>
    <figcaption><center><b>2種分類の出力層</b></center></figcaption>
<figure>
    
<br>

たとえば、上記のような出力があったとします。このクロスエントロピー誤差を計算すると以下のようになります。<br>


$$
E = -1\log0.8 - 0\log0.2 = 0.097 + 0 = 0.097
$$


つまり、$y_i$が1に近づくほど$logy_i$が0に近づくので誤差が小さくなり、yが0に近づくほど$logy_i$が-∞に近づくので誤差が大きくなるという原理であることがわかります。

#### 問題

- クロスエントロピー誤差について正しい選択肢を選んでください。


- 正解ラベルと予測ラベルの値が近いほど小さい値となる。
- 多クラス分類に特化した誤差関数である。
- 回帰問題に頻繁に用いられる誤差関数である。

#### ヒント

- クロスエントロピーは二値分類の評価に特化しており、主に分類モデルの誤差関数として使われます。

#### 解答

- 正解ラベルと予測ラベルの値が近いほど小さい値となる。

***

## 2.6 最適化関数

### 2.6.1 最適化関数

誤差関数で微分した値を元に、方向と程度を決めて重みを更新します。<br>
その際、 **学習率、エポック数、過去の重みの更新量など** をどのように重みの更新に反映するかを定めるために使用するのが <b style='color: #AA0000'>最適化関数</b>です。
<a href='https://aidemy.net/courses/5090/exercises/HkMP93IoLlM' target='_blank'>学習率</a>、<a href='https://aidemy.net/courses/5090/exercises/Sy4v9n8oUxz' target='_blank'>エポック数</a>についての詳細は「ディープランニングの基礎」で説明しています。<br>

<img src='https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_40.gif' width=500>

**<center>図2.6.1-1 最適化関数</center>**

最適化関数は人が調整する必要のある **ハイパーパラメータ** です。<br>
図の通り、最適化関数にはいくつかの種類があり、正しく選択しないと **学習に悪影響**を及ぼす可能性があるので注意が必要です。

> 出典: <a href='http://cs231n.github.io/neural-networks-3/#add' target='_blank'>http://cs231n.github.io/neural-networks-3/#add</a>

#### 問題

 - 最適化関数の性質を示した文として適切なものを選んでください。

- 一般に、最適化関数を最大化するように各層の重みを更新する。
- 最適化関数はどれを選んでも最適化されるため、選ぶ必要はない。
- 最適化関数は損失関数、エポック数など複数の情報を踏まえて重みの更新を行う。
- 最適化関数には1種類しかないので、これはハイパーパラメータではない。

#### ヒント

- 最適化関数は様々な要素を踏まえて重みの更新を行いますが、手法によって重みの更新の仕方が異なり、ハイパーパラメータの一種となっています。

#### 解答

- 最適化関数は損失関数、エポック数など複数の情報を踏まえて重みの更新を行う。

***

## 2.7 学習率

### 2.7.1 学習率

<b style='color: #AA0000'>学習率</b>とは、 **各層の重みを一度にどの程度変更するかを決めるハイパーパラメータ** です。

下図は、最小化を行うモデルと、 **学習率** が与える影響を図示したものです。右上の点が初期値です。

<img src='https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5090_dnn/dnn_chap2_50.png'>

**<center>図2.7.1-1 学習率</center>**

1. 学習率が低すぎて、ほとんど更新が進んでいません。
1. 適切な学習率により、少ない回数で値が収束しています。
1. 収束はしますが、値が大きいため、更新に無駄があります。
1. 学習率が高すぎて、値が発散しています。(上側に更新され、値がどんどん大きくなっています)

つまり、モデルの学習を適切に行うためには、 **損失関数に対する適切な学習率を設定する必要** があるのです。

#### 問題

- 次の3つのうち、一番精度の高い学習率を予想し、コードの一部を変更してください。
- 学習率がモデルの学習に与える影響を確認しましょう。
  - `funcA() lr: 0.01`
  - `funcB() lr: 0.1`
  - `funcC() lr: 1.0`
- A、B、Cのなかから一つ選んで残し、それ以外の2行をコメントアウトしてください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))


def funcA():
    global lr
    lr = 0.01

def funcB():
    global lr
    lr = 0.1

def funcC():
    global lr
    lr = 1.0

# 3つのうち1つを選び、他の2行をコメントアウトして学習率を決めます。
#---------------------------
funcA()
funcB()
funcC()
#---------------------------

sgd = optimizers.SGD(lr=lr)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 全てのパターンを試してみましょう。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))


def funcA():
    global lr
    lr = 0.01

def funcB():
    global lr
    lr = 0.1

def funcC():
    global lr
    lr = 1.0

# 3つのうち1つを選び、他の2行をコメントアウトして学習率を決めます。
#---------------------------
#funcA()
funcB()
#funcC()
#---------------------------

sgd = optimizers.SGD(lr=lr)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.8 ミニバッチ学習

### 2.8.1 ミニバッチ学習

モデルに一度に入力するデータの数を<b style='color: #AA0000'>バッチサイズ</b>と呼びますが、これも **ハイパーパラメータ** のひとつです。<br>
一度に複数のデータを渡した場合、モデルはデータごとに損失と損失関数の **勾配** を計算し、それぞれのデータの勾配の平均値をもとに1度だけ重みの更新をします。<br>

このように、 **複数のデータ** で **重みの更新**を行うことで **偏ったデータの影響を減らし** 、並列計算を行うことで **計算時間を短縮** することもできます。<br>
しかし一方で、大きな重みの更新が発生しにくくなり、 **一部のデータに最適化されてしまい、全体のデータへの最適化が行われなくなる状態（局所解）から抜け出せなくなる可能性** もあります。<br>

それを回避するためには、イレギュラーなデータが多い時には **バッチサイズを大きく**する、少ないときには **バッチサイズを小さくする** といったように、バッチサイズを調整します。<br>

バッチサイズを **1** に設定する学習法を <b style='color: #AA0000'>オンライン学習</b>(確率的勾配降下法)、 **全データ数** に設定する学習方法を <b style='color: #AA0000'>バッチ学習</b>(最急降下法）と呼びます。
また、その **中間** となる少ない数に設定する学習法を <b style='color: #AA0000'>ミニバッチ学習</b>と呼びます。

#### 問題

- 次の3つのうち一番精度の高いバッチサイズを予想し、コードの一部を変更してください。
- バッチサイズがモデルの学習に与える影響を確認しましょう。
  - `funcA() batch_size: 16`
  - `funcB() batch_size: 32`
  - `funcC() batch_size: 64`
- A、B、Cのなかから一つ選んで残し、それ以外の2行をコメントアウトしてください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global batch_size
    batch_size = 16

def funcB():
    global batch_size
    batch_size = 32

def funcC():
    global batch_size
    batch_size = 64

# 3つのうち1つを選び、他の2行をコメントアウトしてbatch_sizeを決めてください。
#---------------------------
# batch_size: 16
funcA()
# batch_size: 32
funcB()
#batch_size: 64
funcC()
#---------------------------

model.fit(X_train, y_train, batch_size=batch_size, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 全てのパターン実行してみましょう。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global batch_size
    batch_size = 16

def funcB():
    global batch_size
    batch_size = 32

def funcC():
    global batch_size
    batch_size = 64

# 3つのうち1つを選び、他の2行をコメントアウトしてbatch_sizeを決めてください。
#---------------------------
#funcA()
#funcB()
funcC()
#---------------------------

model.fit(X_train, y_train, batch_size=batch_size, epochs=3, verbose=1)

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

## 2.9 反復学習

### 2.9.1 反復学習

一般的にディープランニングでは<b style='color: #AA0000'>反復学習</b>を行い、**同じ訓練データで学習を繰り返します。**<br>
学習の回数を **エポック数** といいますが、これもハイパーパラメータです。
エポック数は多く設定すれば、モデルの精度が上がり続けるというものではありません。<br>

適切なエポック数を設定しなかった場合、途中から精度が伸びなくなり、それだけでなく学習を繰り返すことで損失関数を最小化させようとして<a href='https://aidemy.net/courses/2010/exercises/rJ9Yn8iIlM' target='_blank'>過学習</a>を引き起こしてしまう可能性があります。<br> 

そのため、適切なエポック数を設定し、タイミング良く **学習を打ち切る** ことも重要なのです。

#### 問題

- 次の3つのうち一番精度の高いエポック数を予想し、コードの一部を変更してください。
- エポック数がモデルの学習に与える影響を確認しましょう。
  - `funcA() epochs: 5`
  - `funcB() epochs: 10`
  - `funcC() epochs: 60`
- A、B、Cのなかから一つ選んで残し、それ以外の2行をコメントアウトしてください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:1500]
X_test = X_test.reshape(X_test.shape[0], 784)[:6000]
y_train = to_categorical(y_train)[:1500]
y_test = to_categorical(y_test)[:6000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
# 今回はDropoutを使いません。
#model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global epochs
    epochs = 5

def funcB():
    global epochs
    epochs = 10

def funcC():
    global epochs
    epochs = 60

# 3つのうち1つを選び、他の2行をコメントアウトしてエポック数を決めてください。
#---------------------------
# epochs: 5
funcA()
# epochs: 10
funcB()
# epochs: 60
funcC()
#---------------------------

history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 全てのパターンを実行してみましょう。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:1500]
X_test = X_test.reshape(X_test.shape[0], 784)[:6000]
y_train = to_categorical(y_train)[:1500]
y_test = to_categorical(y_test)[:6000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
# 今回はDropoutを使いません。
#model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)

model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

def funcA():
    global epochs
    epochs = 5

def funcB():
    global epochs
    epochs = 10

def funcC():
    global epochs
    epochs = 60

# 3つのうち1つを選び、他の2行をコメントアウトしてエポック数を決めてください。
#---------------------------
#funcA()
funcB()
#funcC()
#---------------------------

history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

***

### このコースのまとめ

- このコースのまとめ
- 次にオススメのコース


【バーチャルYouTuber版動画】<br>
この動画は、バーチャルYouTuber版の動画も配信されています。<br>
https://www.youtube.com/watch?v=YWCbzfqt2GU

#### 問題

- 次の動画をみてください。

https://aidemystorageprd.blob.core.windows.net/movies/5090-sG2C2U7huf.m3u8

Premium Planを受講している方は以下のリンクから受講してください。<br>
https://aidemy.net/courses/5090/exercises/sG2C2U7huf

***

## 2.10 まとめ問題(提出不要)

chapter2ではハイパーパラメータや、適切なディープニューラルネットワークの構成、調整について学習しました。<br>
まとめ問題ではハイパーパラメータのチューニングでMNIST分類の精度向上を行い、より理解を深めましょう。

#### 問題

下記の条件にそって、MNISTの分類モデルをディープニューラルネットワークで実装しましょう。
- テストデータによる正解率は87%以上
- テストデータ、訓練データの正解率の差は1%未満
- エポック数は5
- X_train, y_train, X_test, y_test の定義文は変更不可

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:10000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:10000]

#---------------------------
#  ここに条件にそったコードを書いて下さい       
#---------------------------

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

#### ヒント

- 2.1.1のコードを参考にしみましょう。コード中のハイパーパラメータを１つ変更することで正解率が87%になります。
- 重みの初期化の仕様上、全く同じ値にはなりませんが、ほぼ毎回87%以上の正解率が出せる状態を目指します。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:6000]
X_test = X_test.reshape(X_test.shape[0], 784)[:10000]
y_train = to_categorical(y_train)[:6000]
y_test = to_categorical(y_test)[:10000]

#---------------------------
model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation("sigmoid"))
model.add(Dropout(rate=0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

sgd = optimizers.SGD(lr=0.1)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1)
#---------------------------

score = model.evaluate(X_test, y_test, verbose=0)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

深層学習ではハイパーパラメータが数多くあるので、チューニングに大きな労力を必要とします。その際には既存のモデルなどを参考にすると良いでしょう。

***